# RateML

### Using RateML to generate a CUDA model file and Pyunicore to excute parameter sweeps on HPC (JUSUF cluster)

In [8]:
# !git clone https://github.com/DeLaVlag/tvb-root.git
from IPython.display import Markdown, display, Code

In [19]:
# install tvb-library
%cd tvb-root/scientific_library/
!python setup.py install
# install tvb-bin
# %cd ../tvb_bin
# !python setup.py install
# install tvb-data
# ! pip install tvb-data
%cd ../../

/mnt/user/drive/Shared with groups/RateML TVB/tvb-root/scientific_library
running install
running bdist_egg
running egg_info
creating tvb_library.egg-info
writing tvb_library.egg-info/PKG-INFO
writing dependency_links to tvb_library.egg-info/dependency_links.txt
writing requirements to tvb_library.egg-info/requires.txt
writing top-level names to tvb_library.egg-info/top_level.txt
writing manifest file 'tvb_library.egg-info/SOURCES.txt'
reading manifest file 'tvb_library.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'tvb_library.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying tvb/rateML/XML2model.py -> build/lib/tvb/rateML
copying tvb/rateML/generatedModels/kuramoto.c -> build/lib/tvb/rateML/generatedModels
copying tvb/rateML/generatedModels/rwongwang.c -> build/lib/tvb/rateML/generatedModels
copying tvb/rateML/generatedModels/.ipynb_checkpoints/rwongwang-checkpoint.c -> buil

## Building a model
Building rate based models in RateML, start by creating an XML model file. To understand which constructs can be used to build the model, one should take a closer look at the README file. The cell below will prints the latest README file from the repository. Every construct which can be used, is explained.

In [ ]:
mdfile = open("tvb-root/scientific_library/tvb/rateML/README.md","r")
model = mdfile.read()
display(Markdown(model))
mdfile.close()

In [20]:
# %cd ../../../
!pwd

/mnt/user/drive/Shared with groups/RateML TVB


## Generate a model
After reading the README, one should be able to build an XML model file. Lets use the relatively small Kuramoto model as an example. 
Your model should look like similar to the Kuramoto python file and define some constants, an exposure and dynamics behavior. The dynamics for the Kuramoto consist of a state variable, a derived variable and a time derivative. Except for the derived variable, there are the construct that a RateML XML model file should contain. The template: tvb-root/scientific_library/tvb/rateML/XMLmodels/model_template.xml is an empty template which can be used to create a model XML file.

In [11]:
# Open the Kuramoto model
model_filename = "kuramoto"
model_location = "tvb-root/scientific_library/tvb/rateML/XMLmodels/"+model_filename+".xml"
!pwd
xmlfile = open(model_location,"r")
model = xmlfile.read()
display(Markdown(model))
xmlfile.close()

/mnt/user/drive/Shared with groups/RateML TVB


<Lems description="Kuramoto Base type of any cell/population which has a (dimensionless) rate _R.">

    <ComponentType name="derivatives">

        <!-- Parameters will be assumed to be swept. sweep data will be matched by order of entry -->
        <Parameter name="global_speed" dimension='float'/>
        <Parameter name="global_coupling" dimension='float'/>

        <DerivedParameter name="rec_n" value="1.0f / n_node"/>
        <DerivedParameter name="rec_speed_dt" value="1.0f / global_speed / (dt)"/>
        <DerivedParameter name="nsig" value="sqrt(dt) * sqrt(2.0666 * 1e-5)"/>

        <Constant name="omega" value="60.0 * 2.0 * 3.1415927 / 1e3" dimension=""  
                  description="sets Kuramoto oscillator base line frequency [rad/ms]"/>

        <Exposure name="V" dimension="sin(V)" />

        <Dynamics>
            <!-- Use PI to select the PI wrapper (ie Kuramoto) -->
            <StateVariable name="V" dimension="" exposure="0, PI_2"/>

            <TimeDerivative variable="dV" value="omega + c_pop1"/>
        </Dynamics>
    </ComponentType>

    <ComponentType name="coupling_function">

        <Parameter name="V_j" dimension='0'/>
        <DerivedParameter name="c_pop1" value="global_coupling * rec_n"/>

        <Constant name="a" value="1" dimension="" description="Rescales the connection strength."/>

        <Dynamics>
            <DerivedVariable name="pre" value="sin(V_j - V)"/>
            <DerivedVariable name="post" value="a"/>
        </Dynamics>
    </ComponentType>

    <ComponentType name="noise"/>
</Lems>


In [3]:
%cd ../../../

/mnt/user/drive/Shared with groups/RateML TVB


## Generating the model code

We will call the templating function in order to automatically generate the model code.
In XML2model.py the class 
```python
RateML('model_filename', language=('python' | 'cuda'), 'path/to/your/XMLmodels', 'path/to/your/generatedModels')
``` 
will start the code generation.

In [6]:
from tvb.rateML.XML2model import RateML

# some preexisting examples:
# model_filename = 'montbrio'
# model_filename = 'oscillator'
model_filename = 'kuramoto'
# model_filename = 'rwongwang'
# model_filename = 'epileptor'
language = "cuda"
XMLin = "tvb-root/scientific_library/tvb/rateML/XMLmodels/"
GenModOut = "tvb-root/scientific_library/tvb/rateML/generatedModels/"
RateML(model_filename, language, XMLin, GenModOut)

2020-11-20 16:21:13,230 - INFO - tvb.rateML.XML2model - True validation of tvb-root/scientific_library/tvb/rateML/XMLmodels/kuramoto.xml against https://raw.githubusercontent.com/DeLaVlag/tvb-root/xsdvalidation/scientific_library/tvb/rateML/rML_v0.xsd


In [9]:
# Open the generated model
model_location = "tvb-root/scientific_library/tvb/rateML/generatedModels/"+model_filename+".c"
genModFile = open(model_location,"r")
model = genModFile.read()
display(Code(model, language='c'))
# display(Markdown(model))
genModFile.close()

#include <stdio.h> // for printf
#define PI_2 (2 * M_PI_F)

// buffer length defaults to the argument to the integrate kernel
// but if it's known at compile time, it can be provided which allows
// compiler to change i%n to i&(n-1) if n is a power of two.
#ifndef NH
#define NH nh
#endif

#ifndef WARP_SIZE
#define WARP_SIZE 32
#endif

#include <curand_kernel.h>
#include <curand.h>
#include <stdbool.h>

__device__ float wrap_it_PI(float x)
{
    bool neg_mask = x < 0.0f;
    bool pos_mask = !neg_mask;
    // fmodf diverges 51% of time
    float pos_val = fmodf(x, PI_2);
    float neg_val = PI_2 - fmodf(-x, PI_2);
    return neg_mask * neg_val + pos_mask * pos_val;
}
__device__ float wrap_it_V(float V)
{
    float Vdim[] = {0, PI_2};
    if (V < Vdim[0]) V = Vdim[0];
    else if (V > Vdim[1]) V = Vdim[1];

    return V;
}

__global__ void kuramoto(

        // config
        unsigned int i_step, unsigned int n_node, unsigned int nh, unsigned int n_step, unsigned int n_params,
        float dt, float speed, float * __restrict__ weights, float * __restrict__ lengths,
        float * __restrict__ params_pwi, // pwi: per work item
        // state
        float * __restrict__ state_pwi,
        // outputs
        float * __restrict__ tavg_pwi
        )
{
    // work id & size
    const unsigned int id = (gridDim.x * blockDim.x * threadIdx.y) + threadIdx.x;
    const unsigned int size = blockDim.x * blockDim.y * gridDim.x * gridDim.y;

#define params(i_par) (params_pwi[(size * (i_par)) + id])
#define state(time, i_node) (state_pwi[((time) * 1 * n_node + (i_node))*size + id])
#define tavg(i_node) (tavg_pwi[((i_node) * size) + id])

    // unpack params
    // These are the two parameters which are usually explore in fitting in this model
    const float global_speed = params(0);
    const float global_coupling = params(1);

    // regular constants
    const float omega = 60.0 * 2.0 * 3.1415927 / 1e3;

    // coupling constants, coupling itself is hardcoded in kernel
    const float a = 1;

    // coupling parameters
    float c_pop1 = 0.0;

    // derived parameters
    const float rec_n = 1.0f / n_node;
    const float rec_speed_dt = 1.0f / global_speed / (dt);
    const float nsig = sqrt(dt) * sqrt(2.0666 * 1e-5);


    curandState crndst;
    curand_init(id * (blockDim.x * gridDim.x * gridDim.y), 0, 0, &crndst);

    float V = 0.0;

    float dV = 0.0;

    //***// This is only initialization of the observable
    for (unsigned int i_node = 0; i_node < n_node; i_node++)
    {
        tavg(i_node) = 0.0f;
        if (i_step == 0){
            state(i_step, i_node) = 0.001;
        }
    }

    //***// This is the loop over time, should stay always the same
    for (unsigned int t = i_step; t < (i_step + n_step); t++)
    {
    //***// This is the loop over nodes, which also should stay the same
        for (int i_node = 0; i_node < n_node; i_node++)
        {
            c_pop1 = 0.0f;

            V = state((t) % nh, i_node + 0 * n_node);

            // This variable is used to traverse the weights and lengths matrix, which is really just a vector. It is just a displacement. /
            unsigned int i_n = i_node * n_node;

            for (unsigned int j_node = 0; j_node < n_node; j_node++)
            {
                //***// Get the weight of the coupling between node i and node j
                float wij = weights[i_n + j_node]; // nb. not coalesced
                if (wij == 0.0)
                    continue;

                // Get the delay between node i and node j
                unsigned int dij = lengths[i_n + j_node] * rec_speed_dt;

                //***// Get the state of node j which is delayed by dij
                float V_j = state(((t - dij + nh) % nh), j_node + 0 * n_node);

                // Sum it all together using the coupling function. Kuramoto coupling: (postsyn * presyn) == ((a) * (sin(xj - xi))) 
                c_pop1 += wij * a * sin(V_j - V);

            } // j_node */

            // rec_n is 

## Simulating the result
If the model displays all its features to your whishes, it is time to take her for a spin on a GPU on the JUSUF cluster using PyUnicore. This will only work if you have an LDAP account and you are registered in the PyUnicore database. If you dont have access any other CUDA enabled GPU, will run your generated model.

### Setup PyUnicore

In [11]:
!pip install pyunicore --upgrade
import pyunicore.client as unicore_client
import json
import os

Requirement already up-to-date: pyunicore in /opt/app-root/lib/python3.6/site-packages (0.9.1)
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


In [12]:
token = clb_oauth.get_token()
tr = unicore_client.Transport(token)
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)
# r.site_urls
# HPC_LOC = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core"
HPC_LOC = 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core'
site = unicore_client.Client(transport=tr,site_url=HPC_LOC)

### Transfer model
Transfer the generated model file to JUSUF.

In [13]:
base_url = HPC_LOC + "/storages/PROJECT/"
# base_url = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/storages/PROJECT/"
source_location = "drive/Shared with groups/RateML TVB/tvb-root/scientific_library/tvb/rateML/generatedModels/" + model_filename + ".c"
source_path = os.path.join(os.environ['HOME'], source_location)

storage = unicore_client.Storage(tr, base_url)
storage_location = "wikicollab/RateML/" + model_filename + ".c"
storage.upload(source_path, destination = storage_location)

### Job setup

In [14]:
my_job = {}

# executable / application
# arguments for runthingsJusuf are: backend modelname couplings speeds
my_job['Executable'] = "source /p/project/cslns/wikicollab/RateML/activate; \
cd /p/project/cslns/wikicollab/RateML/; \
./runthingsJuwels " + model_filename + " 32 32;"
# ./runthingsJusuf " + model_filename + " 32 32;"

# environment vars
# run this on login node, not in batch system
my_job['RunOnLoginNode'] = "true"

my_job['Job type'] = "interactive"

# data stage in - TBD
my_job['Imports'] = []

# data stage out - TBD
my_job['Exports'] = []

# Resources - TBD
my_job['Resources'] = {}

### Submit Job to JUSUF 

In [15]:
job = site.new_job(job_description=my_job)

### Info about JOB

In [ ]:
if(job.is_running()):
    print('Job is running')

job.poll()
print('Job is finished')

Job is running


### Fetch results
Copy the output log from JUSUF to Collab

In [ ]:
remote = storage.stat("wikicollab/RateML/output.out")
remote.download("output.out")

with open("output.out", "r") as f:
    for line in f:
        print (line.rstrip())

Copy the error log from JUSUF to Collab

In [ ]:
remote = storage.stat("wikicollab/RateML/error.er")
remote.download("error.er")

with open("error.er", "r") as f:
    for line in f:
        print (line.rstrip())